<a href="https://colab.research.google.com/github/Nastpavlova/python-urban/blob/main/%D0%97%D0%B0%D1%87%D1%91%D1%82%D0%BD%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [65]:
#установим необходимое для работы
!pip install folium -U
!pip install geopandas mapclassify osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
#подключим необходимые для работы библиотеки, которые не входят в стандартный набор

import pandas as pd
import geopandas as gpd
import osmnx as ox

In [67]:
#константа, чтобы везде была одна и таже подложка карт
TILES = "CartoDB positron"

#данные, с которыми будет идти работа в проекте, в одном месте
TERRITORY_NAME = 'Московский район, Санкт-Петербург'
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
KGIOP_FILE_URL = "https://raw.githubusercontent.com/Nastpavlova/python-urban/main/kgiop_objects.geojson"

In [68]:
def get_google_drive_download_url(url: str) -> str: #Функция возвращает ссылку для скачивания с google drive
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

# Территория

## Загрузка территории из OSM (Extract)

In [69]:
#выбрала Московский раойн, далее выгружу его из OpenStreetMap и визуализирую 
territory = ox.geocode_to_gdf(TERRITORY_NAME)  #датафрейм с геотмерией Московского района
territory.explore(color='purple', tiles=TILES) #визуализируем данные в фиолетовом цвете

# Улицы

## Загрузка файла с улицами из google disk (Extract)

In [70]:
url = get_google_drive_download_url(STREETS_FILE_URL) #файл с улицами из google disk мы прописывали в начале (STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing")
gdf_streets = gpd.read_file(url, mask=territory)  #фильтруем улицы по маске геометрии территории
gdf_streets

,type,id,tags,geometry
0,way,4454679,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3376114.437 8372845.795, 3376115.2..."
1,way,4455257,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373432.261 8367053.679, 3373385.1..."
2,way,4456250,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373190.720 8367201.523, 3373256.3..."
3,way,4456529,"{'highway': 'trunk', 'int_ref': 'E 95', 'lanes...","LINESTRING (3375634.027 8355843.811, 3375635.6..."
4,way,4457455,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3379846.768 8367371.582, 3379829.6..."
...,...,...,...,...
10657,way,983206165,"{'highway': 'residential', 'name': '26-я линия'}","LINESTRING (3367405.669 8360587.214, 3367396.6..."
10658,way,983235002,{'highway': 'service'},"LINESTRING (3371676.463 8361986.839, 3371707.2..."
10659,way,983235003,{'highway': 'service'},"LINESTRING (3371843.309 8361636.365, 3371869.1..."
10660,way,983236061,"{'access': 'private', 'highway': 'service'}","LINESTRING (3370177.190 8363516.976, 3370183.2..."


# Обработка данных с улицами (Transform)

С помощью json_normalize преобразовываю вложенные структуры данных в json файле в таблицу.

In [71]:
tags = pd.json_normalize(gdf_streets["tags"])
tags

,foot,highway,lanes,lit,maxspeed,name,oneway,surface,parking:lane:right,trolley_wire,...,lanes:forward,turn:lanes:forward,maxheight:hgv,previous:vehicle,construction,source:name,turn:backward,turn:forward,proposed:oneway,construction:bridge
0,no,residential,2,yes,RU:urban,улица Севастьянова,yes,asphalt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no,secondary,2,yes,RU:urban,Краснопутиловская улица,yes,asphalt,no_stopping,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no,secondary,2,yes,RU:urban,Краснопутиловская улица,yes,asphalt,no_stopping,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,trunk,5,yes,RU:urban,Пулковское шоссе,yes,asphalt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no,tertiary,2,yes,RU:urban,Витебский проспект,NaN,asphalt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10657,NaN,residential,NaN,NaN,NaN,26-я линия,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10658,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10659,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10660,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Columns: 106 entries, foot to construction:bridge
dtypes: object(106)
memory usage: 8.6+ MB


In [73]:
#Из всех тегов оставим только теги name и name:ru.
extract_tags = [
    "name", 
]

#прицепляю теги к улицам с помощью метода join
gdf_streets = gdf_streets.join(tags[extract_tags])
gdf_streets

,type,id,tags,geometry,name
0,way,4454679,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3376114.437 8372845.795, 3376115.2...",улица Севастьянова
1,way,4455257,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373432.261 8367053.679, 3373385.1...",Краснопутиловская улица
2,way,4456250,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373190.720 8367201.523, 3373256.3...",Краснопутиловская улица
3,way,4456529,"{'highway': 'trunk', 'int_ref': 'E 95', 'lanes...","LINESTRING (3375634.027 8355843.811, 3375635.6...",Пулковское шоссе
4,way,4457455,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3379846.768 8367371.582, 3379829.6...",Витебский проспект
...,...,...,...,...,...
10657,way,983206165,"{'highway': 'residential', 'name': '26-я линия'}","LINESTRING (3367405.669 8360587.214, 3367396.6...",26-я линия
10658,way,983235002,{'highway': 'service'},"LINESTRING (3371676.463 8361986.839, 3371707.2...",NaN
10659,way,983235003,{'highway': 'service'},"LINESTRING (3371843.309 8361636.365, 3371869.1...",NaN
10660,way,983236061,"{'access': 'private', 'highway': 'service'}","LINESTRING (3370177.190 8363516.976, 3370183.2...",NaN


In [74]:
#удалим те улицы, у которых нет названия.
gdf_streets.dropna(
    how="all",  # удалить если отсутствуют значения во всех указанных столбцах
    subset=["name"],  # столбцы, в которых надо искать пропуски
    inplace=True
)
gdf_streets

,type,id,tags,geometry,name
0,way,4454679,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3376114.437 8372845.795, 3376115.2...",улица Севастьянова
1,way,4455257,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373432.261 8367053.679, 3373385.1...",Краснопутиловская улица
2,way,4456250,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3373190.720 8367201.523, 3373256.3...",Краснопутиловская улица
3,way,4456529,"{'highway': 'trunk', 'int_ref': 'E 95', 'lanes...","LINESTRING (3375634.027 8355843.811, 3375635.6...",Пулковское шоссе
4,way,4457455,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3379846.768 8367371.582, 3379829.6...",Витебский проспект
...,...,...,...,...,...
10643,way,980878664,"{'highway': 'residential', 'name': 'Средняя Са...","LINESTRING (3369803.914 8362270.100, 3369811.6...",Средняя Садовая улица
10644,way,980878665,"{'highway': 'residential', 'name': 'Косая алле...","LINESTRING (3369844.356 8362212.530, 3369859.1...",Косая аллея
10645,way,980878666,"{'highway': 'residential', 'name': 'Речная алл...","LINESTRING (3369979.887 8362045.249, 3369962.8...",Речная аллея
10649,way,981974253,"{'highway': 'unclassified', 'lanes': '1', 'max...","LINESTRING (3373466.982 8364365.242, 3373469.9...",2-й Предпортовый проезд


Проверим наличие дубликатов в "id", "geometry", "name"

In [75]:
gdf_streets.duplicated(subset=["id", "geometry", "name"]).value_counts()

False    1721
dtype: int64

Проверим, являются ли названия улиц уникальными.

In [76]:
gdf_streets["name"].is_unique

False

Сгруппируем геометрии по столбцу name и объединим геометрии в одну с помощью метода dissolve 

In [77]:
gdf_streets = gdf_streets.dissolve(by="name")
gdf_streets

,geometry,type,id,tags
name,,,,
0-я линия,"MULTILINESTRING ((3369124.063 8361843.816, 336...",way,137816726,{'addr:suburb': 'сдт «Дачное» Балтийского заво...
1-й Предпортовый проезд,"MULTILINESTRING ((3373081.627 8365227.987, 337...",way,35967288,"{'highway': 'residential', 'lanes': '2', 'lit'..."
1-я аллея,"MULTILINESTRING ((3368591.989 8361184.182, 336...",way,93773849,"{'highway': 'service', 'name': '1-я аллея', 's..."
1-я линия,"MULTILINESTRING ((3368791.118 8362213.815, 336...",way,137816727,"{'access': 'private', 'addr:suburb': 'сдт «Дач..."
10-я линия,"MULTILINESTRING ((3368567.811 8361154.814, 336...",way,93773842,{'addr:suburb': 'сдт «Дачное» Балтийского заво...
...,...,...,...,...
улица Севастьянова,"MULTILINESTRING ((3376114.437 8372845.795, 337...",way,4454679,"{'foot': 'no', 'highway': 'residential', 'lane..."
улица Типанова,"MULTILINESTRING ((3376280.682 8367185.099, 337...",way,31399625,"{'cycleway:right': 'share_busway', 'foot': 'no..."
улица Титова,"MULTILINESTRING ((3377213.996 8367358.681, 337...",way,4458329,"{'highway': 'residential', 'lanes': '1', 'lit'..."


In [79]:
gdf_streets.index.rename("Название улицы", inplace=True)
gdf_streets.head()

,geometry,type,id,tags
Название улицы,,,,
0-я линия,"MULTILINESTRING ((3369124.063 8361843.816, 336...",way,137816726,{'addr:suburb': 'сдт «Дачное» Балтийского заво...
1-й Предпортовый проезд,"MULTILINESTRING ((3373081.627 8365227.987, 337...",way,35967288,"{'highway': 'residential', 'lanes': '2', 'lit'..."
1-я аллея,"MULTILINESTRING ((3368591.989 8361184.182, 336...",way,93773849,"{'highway': 'service', 'name': '1-я аллея', 's..."
1-я линия,"MULTILINESTRING ((3368791.118 8362213.815, 336...",way,137816727,"{'access': 'private', 'addr:suburb': 'сдт «Дач..."
10-я линия,"MULTILINESTRING ((3368567.811 8361154.814, 336...",way,93773842,{'addr:suburb': 'сдт «Дачное» Балтийского заво...


Удалим ненужные столбцы ("id", "type" и "tags") из датафрейма с улицами. Оставляем только "название улицы" и "geometry"

In [18]:
drop_columns = [
    "id",
    "type", 
    "tags",
]
gdf_streets.drop(columns=drop_columns, inplace=True)
gdf_streets.head()

,geometry
Название улицы,
0-я линия,"MULTILINESTRING ((3369124.063 8361843.816, 336..."
1-й Предпортовый проезд,"MULTILINESTRING ((3373081.627 8365227.987, 337..."
1-я аллея,"MULTILINESTRING ((3368591.989 8361184.182, 336..."
1-я линия,"MULTILINESTRING ((3368791.118 8362213.815, 336..."
10-я линия,"MULTILINESTRING ((3368567.811 8361154.814, 336..."


In [19]:
gdf_streets.explore(tiles=TILES)

# Объекты культурного наследия

## Загрузка объектов культурного наследия

In [40]:
gdf_kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory)
gdf_kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,478,—,Митрофаниевское кладбище (православное и лютер...,1864-1902,"Митрофаньевское шоссе, д.4, 6, 8, 10, 16 (Митр...",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-97 от 04.03.2014,Ансамбль,POINT (3373155.231 8376583.125)
1,481,Митрофаниевское кладбище (православное и лютер...,Склеп,—,"Митрофаньевское шоссе, д.4, 6, 8, 10, 16",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-97 от 04.03.2014,Памятник,POINT (3372776.188 8376619.079)
2,482,Митрофаниевское кладбище (православное и лютер...,Кладбище,—,"Митрофаньевское шоссе, д.4, 6, 8, 10, 16; квар...",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-97 от 04.03.2014,Памятник,POINT (3373365.514 8376570.252)
3,3622,—,Комплекс построек городского механического хле...,"1915-1916, 1918","Малая Митрофаньевская ул., 4",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3374857.863 8376646.378)
4,3598,—,"Братское захоронение воинов Советской Армии, п...",—,Авиагородок,объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 328 от 03.05....,Памятник,POINT (3372484.309 8358734.316)
...,...,...,...,...,...,...,...,...,...
326,7340,"Сооружения рубежа ""Ижора"" 1943 г.","ДОТ № 75 рубежа ""Ижора""",1943,"ул. Типанова, возле д. 25",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-192 от 07.05.2015,Памятник,POINT (3378330.808 8367347.975)
327,7364,Сооружения периода 1941-1943 гг.,ДОТ артиллерийский,1941-1943,600 м юго-западнее здания по адресу: Московско...,объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-192 от 07.05.2015,Памятник,POINT (3379888.836 8358970.780)
328,9561,—,Две казармы для служащих Варшавского вокзала,—,"Малая Митрофаньевская ул., два дома без номера...",выявленный объект культурного наследия,Распоряжение КГИОП от 29.12.2018 № 560-р,Памятник,POINT (3373659.620 8376605.097)
329,9668,Доты рубежа «Ижора»,Дот №58,1941-1944,"Пулковское шоссе, дом 2, сооружение 2, литера А",выявленный объект культурного наследия,Распоряжение КГИОП № 136-р от 22.04.2021,Памятник,POINT (3375601.588 8364465.593)


In [ ]:
gdf_kgiop_objects.explore(tiles=TILES)

In [42]:
STREET_BUFFER = 100

def get_contains_kgiop_objects(street) -> int:
    """ Функция для подсчета количества объектов культурного наследия попадающих в буффер улицы"""
    return sum(gdf_kgiop_objects["geometry"].within(street.buffer(STREET_BUFFER)))


gdf_streets["contains_kgiop_objects"] = gdf_streets["geometry"].apply(get_contains_kgiop_objects)
gdf_streets.nlargest(5, "contains_kgiop_objects")

,type,id,tags,geometry,name,contains_kgiop_objects
Название улицы,,,,,,
2594,way,142142191,"{'foot': 'no', 'highway': 'primary', 'int_ref'...","LINESTRING (3375215.455 8371228.225, 3375216.9...",Московский проспект,7
2601,way,142147527,"{'foot': 'no', 'highway': 'primary', 'int_ref'...","LINESTRING (3375308.696 8369705.357, 3375297.1...",Московский проспект,5
5527,way,309153689,"{'hgv': 'no', 'highway': 'tertiary', 'lanes': ...","LINESTRING (3374645.532 8375704.648, 3374667.4...",Ташкентская улица,5
5739,way,323757908,"{'addr:postcode': '196084', 'highway': 'unclas...","LINESTRING (3375188.237 8375905.638, 3375369.9...",Черниговская улица,4
9082,way,778228482,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3375043.744 8379349.488, 3375075.3...",набережная Обводного канала,4


In [ ]:
gdf_streets.explore("contains_kgiop_objects", tiles=TILES)

In [ ]:
gdf_streets["density"] = gdf_streets["contains_kgiop_objects"] / gdf_streets.length

select_columns = []
gdf_streets.nlargest(5, "density")

,geometry,contains_kgiop_objects,density
Название улицы,,,
Ташкентская улица,"MULTILINESTRING ((3374645.532 8375704.648, 337...",5,0.007472
площадь Московские Ворота,"MULTILINESTRING ((3375093.549 8375550.330, 337...",2,0.007363
Детский переулок,"MULTILINESTRING ((3375111.349 8376216.777, 337...",3,0.004917
Смоленская улица,"MULTILINESTRING ((3375941.803 8378531.620, 337...",3,0.002542
Малая Митрофаньевская улица,"MULTILINESTRING ((3373001.532 8376648.553, 337...",6,0.002160


In [ ]:
m = gdf_streets.explore("contains_kgiop_objects", tiles=TILES)
m = territory.explore(m=m, style_kwds={"fill": False, "weight": 5})

m

In [43]:
#сохраняю датафрейм gdf_streets в файл streets_with_contains_kgiop.geojson в формате geojson
gdf_streets.to_file('streets_with_contains_kgiop.geojson', driver='GeoJSON')